In [25]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [26]:
main_data = pd.read_csv("Eminem_Lyrics.csv", sep='\t', comment='#', encoding = "ISO-8859-1")
main_data.head()

,Album_Name,Song_Name,Lyrics,Album_URL,Views,Release_date,Unnamed: 6
0,Music To Be Murdered By: Side B,Alfred (Intro),"[Intro: Alfred Hitchcock]\nThus far, this albu...",https://genius.com/albums/Eminem/Music-to-be-m...,24.3K,"December 18, 2020",NaN
1,Music To Be Murdered By: Side B,Black Magic,"[Chorus: Skylar Grey & Eminem]\nBlack magic, n...",https://genius.com/albums/Eminem/Music-to-be-m...,180.6K,"December 18, 2020",NaN
2,Music To Be Murdered By: Side B,Alfreds Theme,"[Verse 1]\nBefore I check the mic (Check, chec...",https://genius.com/albums/Eminem/Music-to-be-m...,285.6K,"December 18, 2020",NaN
3,Music To Be Murdered By: Side B,Tone Deaf,"[Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...",https://genius.com/albums/Eminem/Music-to-be-m...,210.9K,"December 18, 2020",NaN
4,Music To Be Murdered By: Side B,Book of Rhymes,"[Intro]\nI don't smile, I don't frown, get too...",https://genius.com/albums/Eminem/Music-to-be-m...,193.3K,"December 18, 2020",NaN


In [27]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
def intro_words_filter(text):
  
  prepared_text = re.sub(r'\[([^]]*)]', '', text)
  prepared_text = prepared_text.replace("  ", " ")

  return prepared_text

In [29]:
main_data["Prepared_Lyrics"] = main_data["Lyrics"].apply(lambda text: intro_words_filter(text))

main_data

,Album_Name,Song_Name,Lyrics,Album_URL,Views,Release_date,Unnamed: 6,Prepared_Lyrics
0,Music To Be Murdered By: Side B,Alfred (Intro),"[Intro: Alfred Hitchcock]\nThus far, this albu...",https://genius.com/albums/Eminem/Music-to-be-m...,24.3K,"December 18, 2020",NaN,"\nThus far, this album has provided musical ac..."
1,Music To Be Murdered By: Side B,Black Magic,"[Chorus: Skylar Grey & Eminem]\nBlack magic, n...",https://genius.com/albums/Eminem/Music-to-be-m...,180.6K,"December 18, 2020",NaN,"\nBlack magic, night walker (Yeah)\nShe haunts..."
2,Music To Be Murdered By: Side B,Alfreds Theme,"[Verse 1]\nBefore I check the mic (Check, chec...",https://genius.com/albums/Eminem/Music-to-be-m...,285.6K,"December 18, 2020",NaN,"\nBefore I check the mic (Check, check, one, t..."
3,Music To Be Murdered By: Side B,Tone Deaf,"[Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...",https://genius.com/albums/Eminem/Music-to-be-m...,210.9K,"December 18, 2020",NaN,"\nYeah, I'm sorry (Huh?)\nWhat did you say?\nO..."
4,Music To Be Murdered By: Side B,Book of Rhymes,"[Intro]\nI don't smile, I don't frown, get too...",https://genius.com/albums/Eminem/Music-to-be-m...,193.3K,"December 18, 2020",NaN,"\nI don't smile, I don't frown, get too up or ..."
...,...,...,...,...,...,...,...,...
343,Unreleased Songs,Listen To Your Heart,[Chorus: Roxette]\nI know there's something in...,https://genius.com/albums/Eminem/Unreleased-songs,65.5K,NaN,NaN,\nI know there's something in the wake of your...
344,Unreleased Songs,I Get Money (Remix),"[Intro]\nYeah, yeah, I get it\nI run this rap ...",https://genius.com/albums/Eminem/Unreleased-songs,28.5K,NaN,NaN,"\nYeah, yeah, I get it\nI run this rap shit, n..."
345,Unreleased Songs,Cut Back,[Verse]\nI cut back on the syllables just a li...,https://genius.com/albums/Eminem/Unreleased-songs,NaN,2007,NaN,"\nI cut back on the syllables just a little, b..."
346,Unreleased Songs,Hip Hop,[Intro]\nC'mon!\n\n[Verse]\nI still remember t...,https://genius.com/albums/Eminem/Unreleased-songs,NaN,2007,NaN,\nC'mon!\n\n\nI still remember the Hip Hop Sho...


In [30]:
def filtring_punct(text):
  elements = [element if element not in punctuation else '' for element in text]
  return ''.join(elements)

In [31]:
lemmatizer = WordNetLemmatizer()
word_tokenizer = word_tokenize  

In [32]:
main_data["Prepared_Lyrics"] = main_data["Prepared_Lyrics"].apply(lambda text: filtring_punct(text))

In [33]:
lyrics = main_data["Prepared_Lyrics"]
lyrics = lyrics.apply(lambda x: x.replace('\n', ' ').strip())
lyrics

0      Thus far this album has provided musical accom...
1      Black magic night walker Yeah She haunts me li...
2      Before I check the mic Check check one two I g...
3      Yeah Im sorry Huh What did you say Oh I cant h...
4      I dont smile I dont frown get too up or get to...
                             ...                        
343    I know theres something in the wake of your sm...
344    Yeah yeah I get it I run this rap shit now gue...
345    I cut back on the syllables just a little but ...
346    Cmon   I still remember the Hip Hop Shop days ...
347    CDs aint selling no more anyways man Look at S...
Name: Prepared_Lyrics, Length: 348, dtype: object

In [34]:
torch.manual_seed(42)

In [35]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()
model.resize_token_embeddings(len(tokenizer))

loading file https://huggingface.co/gpt2-medium/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/fee58641d7a73348d842afaa337d5a7763dad32beff8d9008bb3c3c847749d6b.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f
loading file https://huggingface.co/gpt2-medium/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/23c853a0fcfc12c7d72ad4e922068b6982665b673f6de30b4c5cbe5bd70a2236.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/gpt2-medium/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/gpt2-medium/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/gpt2-medium/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/gpt2-medium/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3a7a4b7235202f93d14a4a5e8200709184c5b25a29d9cfa6b

Embedding(50259, 1024)

In [36]:
lyrics = lyrics.apply(lambda text: " ".join(text.split(" ")[0:750]))
lyrics

0      Thus far this album has provided musical accom...
1      Black magic night walker Yeah She haunts me li...
2      Before I check the mic Check check one two I g...
3      Yeah Im sorry Huh What did you say Oh I cant h...
4      I dont smile I dont frown get too up or get to...
                             ...                        
343    I know theres something in the wake of your sm...
344    Yeah yeah I get it I run this rap shit now gue...
345    I cut back on the syllables just a little but ...
346    Cmon   I still remember the Hip Hop Shop days ...
347    CDs aint selling no more anyways man Look at S...
Name: Prepared_Lyrics, Length: 348, dtype: object

In [37]:
max_length = max([len(tokenizer.encode(lyric)) for lyric in lyrics])
max_length

1002

In [38]:
class EminemDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [39]:
from torch.utils.data import random_split

In [40]:
dataset = EminemDataset(lyrics, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [41]:
import gc
gc.collect()

92

In [42]:
torch.cuda.empty_cache()

In [43]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

PyTorch: setting up devices


In [44]:
Trainer(model=model,  args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 313
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 313


RuntimeError: ignored

In [ ]:
generated = tokenizer("<|startoftext|>Look, I was gonna go easy on you not to hurt your feelings", return_tensors="pt").input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=50, top_p=0.95, temperature=1.9, num_return_sequences=20)

In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))